In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib3.util.retry import Retry
from fake_useragent import UserAgent
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

user_agent = UserAgent(use_external_data=True)
headers = {'User-Agent': user_agent.random}

base_url = 'https://www.bobaedream.co.kr/cyber/CyberCar.php?sel_m_gubun=ALL&dt_bt=1&priceChk=D&price_txt_1=100&page={}&order=S11&view_size=70'

df_cars=[]

# page번호를 파라미터로 받은 url들을 모아두는 변수
urls=[]

# max페이지넘버를 담는 변수
maxPage= 300

for page in range(1, maxPage):
    url= base_url.format(str(page))
    urls.append(url)

cols=["이름","링크","연식","주행거리","연료","배기량","색상","가격",'위치','무사고']
    
#모든 url마다 실행
for url in urls:
    res=requests.get(url, headers=headers)
    res.raise_for_status() # 응답 상태 코드를 확인하는 함수 | 오류 코드일 경우 예외를 발생시킨다. | 따로 예외처리를 하지 않았으므로 기본적인 예외 처리를 한다.
    requests.adapters.DEFAULT_RETRIES = 100000 # 요청 실패시 재시도 횟수를 설정한다. | 큰 값을 할당하여 실행 시간이 늘어날 수 있다.
    soup=BeautifulSoup(res.text,"lxml")
    
    cars=soup.find_all('p', class_='tit ellipsis')

    links=[]
    #한 url마다 들어있는 모든 차들에 대해 실행
    for car in cars:
        link = "https://www.bobaedream.co.kr" + car.a["href"]
        links.append(link)
        
    for link in links:
        # print(link)

        res2=requests.get(link, headers=headers)
        res2.raise_for_status()
        soup2 = BeautifulSoup(res2.text, "lxml")

        name= soup2.find('h3', class_='tit')
        
    #     # 무사고 여부 | 0 : 무사고
    #     h3_tags = soup2.find_all("h3", class_="tit")

    #     for h3_tag in h3_tags:
    #         img_tag = h3_tag.find_next_sibling("img")
    #         if img_tag is not None and img_tag["src"] == "//file4.bobaedream.co.kr/premium/icon_35.gif":
    #             history = 0
    #         else:
    #             history = 1
        
    #     dl_tag = soup2.find("dl", class_="clearfix")
    #    # dl 태그 안에 있는 dt, dd 태그 선택
    #     dt_tags = dl_tag.find_all("dt")
    #     dd_tags = dl_tag.find_all("dd")

    #     # dt 태그와 dd 태그의 텍스트 출력
    #     for dt_tag, dd_tag in zip(dt_tags, dd_tags):
    #         if dt_tag == '주소':
    #             location = dd_tag.get_text()
    #         else:
    #             location = None
        
        # 표에 있는 내용
        state=soup2.find("div", class_="tbl-01 st-low")
        year=state.find("th",text='연식').find_next_sibling("td")
        km=state.find("th",text='주행거리').find_next_sibling("td")
        fuel=state.find("th",text='연료').find_next_sibling("td")
        amount=state.find("th",text='배기량').find_next_sibling("td")
        color=state.find("th",text='색상').find_next_sibling("td")
        
        price= soup2.find('b', class_='cr')
        
        # print(f'''
        #       이름 : {name.get_text()}
        #       연도 : {year.get_text()}
        #       주행거리 : {km.get_text()}
        #       연료 : {fuel.get_text()}
        #       배기량 : {amount.get_text()}
        #       색상 : {color.get_text()}
        #       가격 : {price.get_text()}
        #       ''')
        
        # cols=["이름","링크","연식","주행거리","연료","배기량","색상","가격"]
        temp=[name.get_text(),link,year.get_text(),km.get_text(),\
          fuel.get_text(),amount.get_text(),color.get_text(), price.get_text()]
        df_cars.append(temp)
df_cars=pd.DataFrame(data=df_cars,columns=cols)
df_cars.to_csv('bobae_cars.csv')
print(df_cars.head())

C:\Users\user\AppData\Local\Temp\ipykernel_11660\1212599231.py:77: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  year=state.find("th",text='연식').find_next_sibling("td")
C:\Users\user\AppData\Local\Temp\ipykernel_11660\1212599231.py:78: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  km=state.find("th",text='주행거리').find_next_sibling("td")
C:\Users\user\AppData\Local\Temp\ipykernel_11660\1212599231.py:79: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  fuel=state.find("th",text='연료').find_next_sibling("td")
C:\Users\user\AppData\Local\Temp\ipykernel_11660\1212599231.py:80: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  amount=state.find("th",text='배기량').find_next_sibling("td")
C:\Users\user\AppData\Local\Temp\ipykernel_11660\1212599231.py:81: DeprecationWarning: The 'text'

AttributeError: 'NoneType' object has no attribute 'get_text'